### Задача идентификации взломщика по его поведению в сети Интернет

Ссылка: [Catch Me If You Can](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2)

У нас есть данные по посещениям пользователями каких-то сайтов и времени посещения.

Необходимо определить сессии в тесте, которые осуществил определенный юзер. Его класс в трейне 1, все остальные юзеры 0.

В ноутбуке приведен алгоритм создания спарс матрицы из сайтов, которые посещали люди из выборки. В каждой строке будет от 1 до 10 непустых элементов.

По времени никаких фич не построено, это для самостоятельной работы.

In [240]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score as auc
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import lil_matrix
from sklearn.model_selection import KFold, GridSearchCV
import time


%matplotlib inline
pd.set_option('display.max_columns', None)

In [241]:
data = pd.read_csv('./data/Alice/train_sessions.csv')
test = pd.read_csv('./data/Alice/test_sessions.csv')

In [260]:
test.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
1,2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
2,3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
3,4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
4,5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [223]:
#запоминаем индекс для последующего разделения трейна и теста
idx_split = data.shape[0]
#объединяем трейн и тест. Создание спарс матрицы в таком, как у нас, виде особо не ликует.
data = data.append(test, sort=False).reset_index(drop=True)

In [224]:
data.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0.0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0.0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0.0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0.0


In [225]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336358 entries, 0 to 336357
Data columns (total 22 columns):
session_id    336358 non-null int64
site1         336358 non-null int64
time1         336358 non-null object
site2         331406 non-null float64
time2         331406 non-null object
site3         326994 non-null float64
time3         326994 non-null object
site4         323503 non-null float64
time4         323503 non-null object
site5         320170 non-null float64
time5         320170 non-null object
site6         317061 non-null float64
time6         317061 non-null object
site7         314137 non-null float64
time7         314137 non-null object
site8         311375 non-null float64
time8         311375 non-null object
site9         308568 non-null float64
time9         308568 non-null object
site10        305858 non-null float64
time10        305858 non-null object
target        253561 non-null float64
dtypes: float64(10), int64(2), object(10)
memory usage: 56.5+ MB


In [226]:
num_cols = [col for col in data.columns if data[col].dtype=='int64' or
                                           data[col].dtype=='float64']
num_cols.remove('target')
time_cols = [col for col in data.columns if data[col].dtype=='object']

In [227]:
for col in time_cols:
    data[col] = pd.to_datetime(data[col], yearfirst=True)

In [228]:
for col in time_cols:
    data[col + 'year'] = data[col].dt.year;
    data[col + 'month'] = data[col].dt.month;
    data[col + 'day'] = data[col].dt.day;
    data[col + 'hours'] = data[col].dt.hour;
    data[col + 'minuts'] = data[col].dt.minute;
    data.drop(col, axis=1, inplace=True)
    

In [229]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336358 entries, 0 to 336357
Data columns (total 62 columns):
session_id      336358 non-null int64
site1           336358 non-null int64
site2           331406 non-null float64
site3           326994 non-null float64
site4           323503 non-null float64
site5           320170 non-null float64
site6           317061 non-null float64
site7           314137 non-null float64
site8           311375 non-null float64
site9           308568 non-null float64
site10          305858 non-null float64
target          253561 non-null float64
time1year       336358 non-null int64
time1month      336358 non-null int64
time1day        336358 non-null int64
time1hours      336358 non-null int64
time1minuts     336358 non-null int64
time2year       331406 non-null float64
time2month      331406 non-null float64
time2day        331406 non-null float64
time2hours      331406 non-null float64
time2minuts     331406 non-null float64
time3year       326994 

In [230]:
num_cols = [col for col in data.columns if data[col].dtype=='int64' or
                                           data[col].dtype=='float64']

#Заполним отсутствующие сайты уникальным значением.
data[num_cols] = data[num_cols].fillna(-1)
#Это необходимо для того, чтобы данные по сайтам привести к целочисленному типу.
data[num_cols] = data[num_cols].astype(int)

In [231]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336358 entries, 0 to 336357
Data columns (total 62 columns):
session_id      336358 non-null int64
site1           336358 non-null int64
site2           336358 non-null int64
site3           336358 non-null int64
site4           336358 non-null int64
site5           336358 non-null int64
site6           336358 non-null int64
site7           336358 non-null int64
site8           336358 non-null int64
site9           336358 non-null int64
site10          336358 non-null int64
target          336358 non-null int64
time1year       336358 non-null int64
time1month      336358 non-null int64
time1day        336358 non-null int64
time1hours      336358 non-null int64
time1minuts     336358 non-null int64
time2year       336358 non-null int64
time2month      336358 non-null int64
time2day        336358 non-null int64
time2hours      336358 non-null int64
time2minuts     336358 non-null int64
time3year       336358 non-null int64
time3month     

In [232]:
data.head()

,session_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target,time1year,time1month,time1day,time1hours,time1minuts,time2year,time2month,time2day,time2hours,time2minuts,time3year,time3month,time3day,time3hours,time3minuts,time4year,time4month,time4day,time4hours,time4minuts,time5year,time5month,time5day,time5hours,time5minuts,time6year,time6month,time6day,time6hours,time6minuts,time7year,time7month,time7day,time7hours,time7minuts,time8year,time8month,time8day,time8hours,time8minuts,time9year,time9month,time9day,time9hours,time9minuts,time10year,time10month,time10day,time10hours,time10minuts
0,1,718,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,2014,2,20,10,2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,2,890,941,3847,941,942,3846,3847,3846,1516,1518,0,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,20,2014,2,22,11,20
2,3,14769,39,14768,14769,37,39,14768,14768,14768,14768,0,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40
3,4,782,782,782,782,782,782,782,782,782,782,0,2014,3,28,10,52,2014,3,28,10,52,2014,3,28,10,53,2014,3,28,10,53,2014,3,28,10,54,2014,3,28,10,54,2014,3,28,10,55,2014,3,28,10,55,2014,3,28,10,56,2014,3,28,10,56
4,5,22,177,175,178,177,178,175,177,177,178,0,2014,2,28,10,53,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,57,2014,2,28,10,57


In [233]:
sites = [col for col in data.columns if col.startswith('site')]

In [234]:
# Этой функцией создаем словарь посещенных юзерами сайтов
def find_sites(li):
    lli = {}
    for l in li:
        if l > 0:
            if l in lli:
                lli[str(l)] += 1
            else:
                lli[str(l)] = 1
    return lli

In [235]:
data['all_sites'] = data[sites].apply(find_sites, axis=1)

In [236]:
data.head()

,session_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target,time1year,time1month,time1day,time1hours,time1minuts,time2year,time2month,time2day,time2hours,time2minuts,time3year,time3month,time3day,time3hours,time3minuts,time4year,time4month,time4day,time4hours,time4minuts,time5year,time5month,time5day,time5hours,time5minuts,time6year,time6month,time6day,time6hours,time6minuts,time7year,time7month,time7day,time7hours,time7minuts,time8year,time8month,time8day,time8hours,time8minuts,time9year,time9month,time9day,time9hours,time9minuts,time10year,time10month,time10day,time10hours,time10minuts,all_sites
0,1,718,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,2014,2,20,10,2,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,{'718': 1}
1,2,890,941,3847,941,942,3846,3847,3846,1516,1518,0,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,19,2014,2,22,11,20,2014,2,22,11,20,"{'890': 1, '941': 1, '3847': 1, '942': 1, '384..."
2,3,14769,39,14768,14769,37,39,14768,14768,14768,14768,0,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,2013,12,16,16,40,"{'14769': 1, '39': 1, '14768': 1, '37': 1}"
3,4,782,782,782,782,782,782,782,782,782,782,0,2014,3,28,10,52,2014,3,28,10,52,2014,3,28,10,53,2014,3,28,10,53,2014,3,28,10,54,2014,3,28,10,54,2014,3,28,10,55,2014,3,28,10,55,2014,3,28,10,56,2014,3,28,10,56,{'782': 1}
4,5,22,177,175,178,177,178,175,177,177,178,0,2014,2,28,10,53,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,55,2014,2,28,10,57,2014,2,28,10,57,"{'22': 1, '177': 1, '175': 1, '178': 1}"


In [239]:
len(data[data['target'] == -999])

82797

In [194]:
from sklearn.metrics import roc_auc_score, accuracy_score

cv = KFold(idx_split, shuffle=True, random_state=777, n_folds=3)
rfc = RandomForestClassifier(n_estimators=15, random_state=777, n_jobs=-1)
params = {'max_features': [4, 7, 10, 13, 17, 22, 27], 'max_depth': [4, 6, 9]}
gs = GridSearchCV(rfc,param_grid=params, cv=cv,scoring='roc_auc')

gs.fit(data[num_cols],  data['target'])
rfc_pred = gs.best_estimator_.predict_proba(data[num_cols][idx_split:])[:, 1]
print(roc_auc_score(data['target'][idx_split:], rfc_pred))

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [173]:
#собираем предсказания теста
answ_df = pd.DataFrame()
for i in range(len(answ)):
    answ_df['an'+str(i)] = answ[i]
answ_df['answer'] = answ_df.mean(axis=1)

In [174]:
answ_df.head()

,an0,an1,an2,an3,an4,an5,an6,an7,an8,an9,answer
0,1.757254e-04,2.409355e-04,1.969017e-04,1.728496e-04,1.380436e-04,1.676958e-04,1.245776e-04,1.563077e-04,1.944676e-04,2.212621e-04,1.788767e-04
1,5.087860e-05,4.553811e-05,4.409239e-05,4.990177e-05,4.370581e-05,3.886130e-05,4.845266e-05,4.634258e-05,4.706723e-05,5.074653e-05,4.655870e-05
2,4.838427e-05,4.125738e-05,4.276697e-05,4.939097e-05,4.651539e-05,4.001193e-05,4.481238e-05,5.306353e-05,5.013573e-05,5.368914e-05,4.700277e-05
3,1.365636e-08,1.263008e-08,1.066219e-08,1.464965e-08,1.352933e-08,1.498467e-08,1.010768e-08,9.379403e-09,1.314464e-08,1.049951e-08,1.232435e-08
4,2.785610e-06,2.659884e-06,2.040587e-07,2.420955e-06,2.437611e-06,1.989232e-06,2.427787e-06,2.945337e-06,2.786196e-06,2.947039e-06,2.360371e-06


In [175]:
#пишем функцию для сабмита и делаем сабмит

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [176]:
y_test = np.array(answ_df['answer'])

In [177]:
write_to_submission_file(y_test, 'submission.csv')